In [2]:
"""
MFC Control System Simulation
=============================
State-space formulation: ẋ = Ax + Bu + d

- A matrix: autonomous dynamics (biological decay/drift)
- B matrix: input coupling (mixing physics)
- d vector: constant drift (zero-order kinetics)

"""

import numpy as np

# =============================================================================
# SYSTEM PARAMETERS
# =============================================================================

# Reactor volume (mL)
V = 1000.0

# Dose volume per pump activation (mL)
DOSE_VOLUME = 5.0

# Simulation time step (minutes)
DT = 1.0

# -----------------------------------------------------------------------------
# Substrate Properties (in the dosing containers)
# -----------------------------------------------------------------------------
SUBSTRATES = {
    'wine': {
        'pH': 4.0,      # 100:1 diluted wine
        'EC': 0.02,     # mS/cm
        'TAN': 0.1,     # mg-N/L
        # Trace metals (mg/L in solution)
        'Fe': 0.05, 'Zn': 0.01, 'Cu': 0.001, 'Mn': 0.02, 'Se': 0.0, 'Ni': 0.0, 'Co': 0.0
    },
    'urine': {
        'pH': 8.5,      # 4:1 diluted fermented urine
        'EC': 6.25,     # mS/cm
        'TAN': 200.0,   # mg-N/L
        'Fe': 0.01, 'Zn': 0.05, 'Cu': 0.01, 'Mn': 0.005, 'Se': 0.001, 'Ni': 0.002, 'Co': 0.001
    },
    'spirulina': {
        'pH': 9.0,      # 15 g/L spirulina solution
        'EC': 1.14,     # mS/cm (from ionic conductivity calculation)
        'TAN': 144.0,   # mg-N/L (from protein content)
        'Fe': 4.275, 'Zn': 0.300, 'Cu': 0.915, 'Mn': 0.285, 'Se': 0.00105, 'Ni': 0.0, 'Co': 0.0
    }
}

# -----------------------------------------------------------------------------
# Control Targets
# -----------------------------------------------------------------------------
TARGETS = {
    'pH': {'min': 6.8, 'max': 7.2, 'setpoint': 7.0},
    'EC': {'min': 5.0, 'max': 10.0, 'setpoint': 7.5},
    'TAN': {'min': 50.0, 'max': 200.0, 'setpoint': 125.0}
}

# -----------------------------------------------------------------------------
# A Matrix: Autonomous Dynamics (state-dependent decay)
# -----------------------------------------------------------------------------
# Small state-dependent terms (first-order kinetics component)
# Units: per minute

A_MATRIX = np.array([
    [-0.0001,  0.0,      0.000002],   # pH: small decay + TAN buffering
    [0.0,      -0.0001,  0.0],         # EC: small decay
    [0.0,      0.0,      -0.00003]     # TAN: small decay
])

# -----------------------------------------------------------------------------
# d Vector: Constant Drift (zero-order kinetics - dominant term)
# -----------------------------------------------------------------------------
# Research shows these rates are approximately constant within operating range
# (Batstone et al., Water Sci Tech 2002; Siegrist et al., Water Sci Tech 2002)

DRIFT = np.array([
    -0.0005,   # pH drifts down ~0.0005/min from CO2/VFA production
    -0.001,    # EC drifts down ~0.001 mS/cm/min from ion uptake
    -0.3       # TAN consumed ~0.3 mg-N/L/min by bacteria
])

# Metal consumption rate (first-order, per minute)
METAL_DECAY_RATE = 0.0005  # ~0.05% per minute

# =============================================================================
# MIXING PHYSICS (B matrix effects)
# =============================================================================

def mix_states(reactor_state, substrate_name, dose_volume, reactor_volume):
    """
    Apply mixing physics when dosing a substrate.

    This implements the B matrix effect: instantaneous CSTR mixing.

    x_new = (x_reactor * V_reactor + x_substrate * V_dose) / (V_reactor + V_dose)
    """
    substrate = SUBSTRATES[substrate_name]
    new_volume = reactor_volume + dose_volume

    # Mix primary states (pH, EC, TAN)
    new_pH = (reactor_state['pH'] * reactor_volume + substrate['pH'] * dose_volume) / new_volume
    new_EC = (reactor_state['EC'] * reactor_volume + substrate['EC'] * dose_volume) / new_volume
    new_TAN = (reactor_state['TAN'] * reactor_volume + substrate['TAN'] * dose_volume) / new_volume

    # Mix trace metals
    new_metals = {}
    for metal in ['Fe', 'Zn', 'Cu', 'Mn', 'Se', 'Ni', 'Co']:
        new_metals[metal] = (reactor_state[metal] * reactor_volume +
                            substrate[metal] * dose_volume) / new_volume

    return {
        'pH': new_pH,
        'EC': new_EC,
        'TAN': new_TAN,
        **new_metals
    }, new_volume

# =============================================================================
# ODE DYNAMICS (A matrix + drift effects)
# =============================================================================

def apply_drift(state, dt):
    """
    Apply autonomous dynamics (biological decay) over time step dt.

    This implements: dx/dt = Ax + d

    Where:
    - Ax: state-dependent decay (first-order, small)
    - d: constant drift (zero-order, dominant)

    Using forward Euler integration: x(t+dt) = x(t) + dx/dt * dt
    """
    # State vector [pH, EC, TAN]
    x = np.array([state['pH'], state['EC'], state['TAN']])

    # dx/dt = Ax + d
    dx_dt = A_MATRIX @ x + DRIFT

    # Euler integration
    x_new = x + dx_dt * dt

    # Apply metal decay (first-order)
    decay_factor = np.exp(-METAL_DECAY_RATE * dt)

    return {
        'pH': max(0, x_new[0]),
        'EC': max(0.01, x_new[1]),
        'TAN': max(0, x_new[2]),
        'Fe': state['Fe'] * decay_factor,
        'Zn': state['Zn'] * decay_factor,
        'Cu': state['Cu'] * decay_factor,
        'Mn': state['Mn'] * decay_factor,
        'Se': state['Se'] * decay_factor,
        'Ni': state['Ni'] * decay_factor,
        'Co': state['Co'] * decay_factor,
    }

# =============================================================================
# CONTROL LAW (Bang-Bang / Relay Control)
# =============================================================================

def control_decision(state):
    """
    Priority-based bang-bang control.

    Priority order:
    1. pH (highest) - bacterial survival
    2. TAN - nitrogen for growth
    3. EC - affects power output
    4. Metals (lowest) - not actively controlled

    Returns: substrate name to dose, or None
    """
    pH = state['pH']
    EC = state['EC']
    TAN = state['TAN']

    # Priority 1: pH control
    if pH > TARGETS['pH']['max']:
        return 'wine'      # Acid to lower pH
    if pH < TARGETS['pH']['min']:
        return 'urine'     # Base to raise pH

    # Priority 2 & 3: TAN and EC combined logic
    if TAN < TARGETS['TAN']['min'] and EC < TARGETS['EC']['min']:
        return 'urine'     # Provides both TAN and EC

    if TAN < TARGETS['TAN']['min']:
        return 'spirulina' # Provides TAN (EC is OK)

    if EC < TARGETS['EC']['min']:
        return 'urine'     # Provides EC (TAN is OK)

    # All in range - no dosing needed
    return None

# =============================================================================
# SIMULATION
# =============================================================================

def run_simulation(duration_minutes=480, initial_state=None):
    """
    Run the MFC control simulation.
    """

    # Default initial state
    if initial_state is None:
        initial_state = {
            'pH': 7.0,
            'EC': 5.5,
            'TAN': 80.0,
            'Fe': 1.0,
            'Zn': 0.1,
            'Cu': 0.05,
            'Mn': 0.1,
            'Se': 0.01,
            'Ni': 0.01,
            'Co': 0.01
        }

    # Initialize
    state = initial_state.copy()
    volume = V
    history = []

    # Counters
    doses = {'wine': 0, 'urine': 0, 'spirulina': 0}
    volumes = {'wine': 0.0, 'urine': 0.0, 'spirulina': 0.0}

    # Print header
    print("=" * 130)
    print("MFC CONTROL SIMULATION")
    print("State-space formulation: ẋ = Ax + Bu + d")
    print("=" * 130)
    print(f"\nA (decay):     diag({A_MATRIX[0,0]:.4f}, {A_MATRIX[1,1]:.4f}, {A_MATRIX[2,2]:.5f})")
    print(f"d (drift):     [{DRIFT[0]:.4f}, {DRIFT[1]:.3f}, {DRIFT[2]:.1f}]")
    print(f"\nTargets: pH [{TARGETS['pH']['min']}-{TARGETS['pH']['max']}], "
          f"EC [{TARGETS['EC']['min']}-{TARGETS['EC']['max']}] mS/cm, "
          f"TAN [{TARGETS['TAN']['min']}-{TARGETS['TAN']['max']}] mg-N/L")
    print("\n" + "=" * 130)
    print(f"\n{'Time':<6} {'pH':>6} {'EC':>8} {'TAN':>8} {'Vol':>8} {'Action':<22} "
          f"{'Fe':>6} {'Zn':>6} {'Cu':>6} {'Mn':>6} {'Se':>7} {'Ni':>7} {'Co':>7}")
    print(f"{'(min)':<6} {'':>6} {'(mS/cm)':>8} {'(mg-N/L)':>8} {'(mL)':>8} {'':<22} "
          f"{'(mg/L)':>6} {'(mg/L)':>6} {'(mg/L)':>6} {'(mg/L)':>6} {'(mg/L)':>7} {'(mg/L)':>7} {'(mg/L)':>7}")
    print("-" * 130)

    for t in range(int(duration_minutes) + 1):
        # Store current state
        history.append({
            'time': t,
            'volume': volume,
            **state.copy()
        })

        # Control decision
        action = control_decision(state)

        # Build action string with reason
        if action == 'wine':
            action_str = "DOSE WINE (pH high)"
        elif action == 'urine':
            if state['pH'] < TARGETS['pH']['min']:
                action_str = "DOSE URINE (pH low)"
            elif state['TAN'] < TARGETS['TAN']['min'] and state['EC'] < TARGETS['EC']['min']:
                action_str = "DOSE URINE (TAN+EC)"
            else:
                action_str = "DOSE URINE (EC low)"
        elif action == 'spirulina':
            action_str = "DOSE SPIRULINA (TAN)"
        else:
            action_str = "---"

        # Print current state
        print(f"{t:<6} {state['pH']:>6.2f} {state['EC']:>8.2f} {state['TAN']:>8.1f} {volume:>8.0f} {action_str:<22} "
              f"{state['Fe']:>6.3f} {state['Zn']:>6.3f} {state['Cu']:>6.3f} {state['Mn']:>6.3f} "
              f"{state['Se']:>7.4f} {state['Ni']:>7.4f} {state['Co']:>7.4f}")

        # Apply control action (mixing physics - B matrix)
        if action:
            state, volume = mix_states(state, action, DOSE_VOLUME, volume)
            doses[action] += 1
            volumes[action] += DOSE_VOLUME

        # Apply drift (ODE dynamics - A matrix + d) for next time step
        if t < duration_minutes:
            state = apply_drift(state, DT)

    # Print summary
    print("-" * 130)
    print("\n" + "=" * 130)
    print("SIMULATION SUMMARY")
    print("=" * 130)

    total_dosed = sum(volumes.values())
    print(f"\nDuration: {duration_minutes} minutes ({duration_minutes/60:.1f} hours)")
    print(f"Final Volume: {volume:.0f} mL")

    print(f"\nSubstrate Usage:")
    for sub in ['wine', 'urine', 'spirulina']:
        pct = (volumes[sub] / total_dosed * 100) if total_dosed > 0 else 0
        print(f"  {sub.capitalize():<12}: {volumes[sub]:>8.0f} mL ({pct:>5.1f}%) - {doses[sub]} doses")
    print(f"  {'Total':<12}: {total_dosed:>8.0f} mL")

    print(f"\nFinal State:")
    print(f"  pH:  {state['pH']:.2f} (target: {TARGETS['pH']['min']}-{TARGETS['pH']['max']})")
    print(f"  EC:  {state['EC']:.2f} mS/cm (target: {TARGETS['EC']['min']}-{TARGETS['EC']['max']})")
    print(f"  TAN: {state['TAN']:.1f} mg-N/L (target: {TARGETS['TAN']['min']}-{TARGETS['TAN']['max']})")

    print(f"\nFinal Trace Metals:")
    metals_targets = {
        'Fe': (1.0, 10.0), 'Zn': (0.1, 1.0), 'Cu': (0.01, 0.5),
        'Mn': (0.1, 1.0), 'Se': (0.01, 0.1), 'Ni': (0.05, 1.0), 'Co': (0.01, 0.3)
    }
    for metal, (lo, hi) in metals_targets.items():
        val = state[metal]
        status = "OK" if lo <= val <= hi else ("LOW" if val < lo else "HIGH")
        print(f"  {metal}: {val:.4f} mg/L (target: {lo}-{hi}) {status}")

    return history


# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    # Run 8-hour simulation
    history = run_simulation(duration_minutes=480)

MFC CONTROL SIMULATION
State-space formulation: ẋ = Ax + Bu + d

A (decay):     diag(-0.0001, -0.0001, -0.00003)
d (drift):     [-0.0005, -0.001, -0.3]

Targets: pH [6.8-7.2], EC [5.0-10.0] mS/cm, TAN [50.0-200.0] mg-N/L


Time       pH       EC      TAN      Vol Action                     Fe     Zn     Cu     Mn      Se      Ni      Co
(min)          (mS/cm) (mg-N/L)     (mL)                        (mg/L) (mg/L) (mg/L) (mg/L)  (mg/L)  (mg/L)  (mg/L)
----------------------------------------------------------------------------------------------------------------------------------
0        7.00     5.50     80.0     1000 ---                     1.000  0.100  0.050  0.100  0.0100  0.0100  0.0100
1        7.00     5.50     79.7     1000 ---                     1.000  0.100  0.050  0.100  0.0100  0.0100  0.0100
2        7.00     5.50     79.4     1000 ---                     0.999  0.100  0.050  0.100  0.0100  0.0100  0.0100
3        7.00     5.50     79.1     1000 ---                     0